<a href="https://colab.research.google.com/github/HUYR01/Federated-Learning-Based-Intrusion-Detection-System/blob/main/FL_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import torch, copy
import argparse, json
import datetime
import os
import logging
import random

logging.getLogger().setLevel(logging.INFO)

no_models=5 #客户端模型数量
global_epochs=5  #全局迭代次数
local_epochs= 3 #局部迭代次数
#k =5  #选择局部模型的个数
batch_size=1024 #本地训练每一轮的样本数
lr = 0.001
momentum= 0.0001
lamda= 0.1

### CNN模型

In [ ]:
import os
import logging

import pandas as pd
import tensorflow.keras as keras

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model

# Log setting
logging.basicConfig(format="%(asctime)s %(levelname)s %(message)s", datefmt="%H:%M:%S", level=logging.INFO)

# Change display.max_rows to show all features.
pd.set_option("display.max_rows", 85)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [ ]:
PROCESSED_DIR_PATH = "/content/drive/My Drive/CICIDS/ProcessedDataset"
MODEL_DIR_PATH = "/content/drive/My Drive/CICIDS/Model"

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler


def preprocessing(df: pd.DataFrame) -> (np.ndarray, np.ndarray):
    # Shuffle the dataset打乱数据集
    df = df.sample(frac=1)

    # 划分特征和标签
    x = df.iloc[:, df.columns != 'Label']
    y = df[['Label']].to_numpy()

    # 将特征归一化到[0,1]区间
    scaler = MinMaxScaler()
    x = scaler.fit_transform(x)

    return x, y


def reshape_dataset_cnn(x: np.ndarray) -> np.ndarray:
    # Add padding columns添加填充列（每一行末尾填充3个零）
    result = np.zeros((x.shape[0], 81))
    result[:, :-3] = x

    # Reshaping dataset
    result = np.reshape(result, (result.shape[0], 9, 9))
    result = result[..., tf.newaxis]#加一个维度
    return result


def plot_history(history: tf.keras.callbacks.History):
    # summarize history for accuracy
    plt.plot(history.history['sparse_categorical_accuracy'])
    plt.plot(history.history['val_sparse_categorical_accuracy'])
    plt.title('model2 accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model2 loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()


def evaluation(model: keras.Model, x_test: np.ndarray, y_test: np.ndarray):
    score = model.evaluate(x_test, y_test, verbose=False)
    logging.info('Evaluation:\nLoss: {}\nAccuracy : {}\n'.format(score[0], score[1]))

    # F1 score
    y_pred = model.predict(x_test, batch_size=1024, verbose=False)
    y_pred = np.argmax(y_pred, axis=1)

    logging.info("\n{}".format(classification_report(y_test, y_pred)))

In [ ]:
def create_cnn_model() -> keras.Model:
    # Creating layers
    inputs = keras.layers.Input(shape=(9, 9, 1))
    x = keras.layers.Conv2D(120, 2, activation='relu', padding="same")(inputs)#卷积核数量120，长宽=2
    x = keras.layers.Conv2D(60, 3, activation='relu', padding="same")(x)
    x = keras.layers.Conv2D(30, 4, activation='relu', padding="same")(x)
    x = keras.layers.Flatten()(x)
    outputs = keras.layers.Dense(15, activation='softmax')(x)#全连接层
    cnn_model = keras.Model(inputs=inputs, outputs=outputs, name='cnn')

    # Compile layers
    cnn_model.compile(loss='sparse_categorical_crossentropy',
                      metrics=['sparse_categorical_accuracy'],
                      optimizer='adam')

    return cnn_model



### 数据读取

In [ ]:
#训练数据读取
df = pd.read_csv(os.path.join(PROCESSED_DIR_PATH, 'train_MachineLearningCVE.csv'), skipinitialspace=True)
logging.info("Class distribution\n{}".format(df.Label.value_counts()))
X, Y = preprocessing(df)
del df
X = reshape_dataset_cnn(X)


INFO:root:Class distribution
0     1818477
4      184858
10     127144
2      102421
3        8235
7        6350
11       4718
6        4637
5        4399
1        1573
12       1205
14        522
9          29
13         17
8           9
Name: Label, dtype: int64


### 联邦学习

现存问题：1、数据划分 2、模型参数更新

In [ ]:
#定义一个服务端和多个客户端
server=create_cnn_model()
weights = server.get_weights() #存储模型参数

print("\nmodel created\n")
logging.info("*** TRAINING START ***")
for i in range(global_epochs):

    print(i,' ')

    data_len=int(len(X) / no_models)

    new_weights=[]#存储本轮模型更新结果
    for weight in weights:
        new_weights.append(np.zeros(weight.shape))

    print('\nlocal_model:\n')
    for c in range(no_models):
      #将客户端模型参数更新为服务端模型参数
      print(c,' ')
      client=create_cnn_model()
      X_c=X[c*data_len:(c+1)*data_len]
      Y_c=Y[c*data_len:(c+1)*data_len]

      #更新客户端模型
      client.set_weights(weights)
      #客户端模型训练
      history=client.fit(X_c, Y_c, validation_split=0.1, epochs=local_epochs, batch_size=batch_size, verbose=True)#0.1为验证集占比


      #模型聚合
      cur_weights=client.get_weights()

      for i in range(len(cur_weights)):
        new_weights[i] += cur_weights[i] / no_models
      print('\nserver:')

    weights = new_weights
    server.set_weights(weights)
    server.save(os.path.join(MODEL_DIR_PATH, "server_weights-06-07.h5"))


server.set_weights(weights)
logging.info("*** TRAINING FINISH ***")

INFO:root:*** TRAINING START ***



model created

0  

local_model:

0  
Epoch 1/3
399/399 [==============================] - 11s 23ms/step - loss: 0.2775 - sparse_categorical_accuracy: 0.9195 - val_loss: 0.1285 - val_sparse_categorical_accuracy: 0.9538
Epoch 2/3
399/399 [==============================] - 9s 22ms/step - loss: 0.1041 - sparse_categorical_accuracy: 0.9571 - val_loss: 0.0985 - val_sparse_categorical_accuracy: 0.9651
Epoch 3/3
399/399 [==============================] - 9s 22ms/step - loss: 0.0841 - sparse_categorical_accuracy: 0.9658 - val_loss: 0.0861 - val_sparse_categorical_accuracy: 0.9629

server:
1  
Epoch 1/3
399/399 [==============================] - 15s 22ms/step - loss: 0.2697 - sparse_categorical_accuracy: 0.9215 - val_loss: 0.1199 - val_sparse_categorical_accuracy: 0.9520
Epoch 2/3
399/399 [==============================] - 9s 23ms/step - loss: 0.1026 - sparse_categorical_accuracy: 0.9578 - val_loss: 0.0957 - val_sparse_categorical_accuracy: 0.9663
Epoch 3/3
399/399 [===========================

INFO:root:*** TRAINING FINISH ***


In [ ]:
#测试数据读取
df_test = pd.read_csv(os.path.join(PROCESSED_DIR_PATH, 'test_MachineLearningCVE.csv'), skipinitialspace=True)
logging.info("Class distribution\n{}".format(df_test.Label.value_counts()))
X_test, Y_test = preprocessing(df_test)
del df_test
X_test = reshape_dataset_cnn(X_test)

INFO:root:Class distribution
0     454620
4      46215
10     31786
2      25606
3       2058
7       1588
11      1179
6       1159
5       1100
1        393
12       302
14       130
9          7
13         4
8          2
Name: Label, dtype: int64


In [ ]:
import warnings
warnings.filterwarnings("ignore")
server.set_weights(weights)
#服务端对当前模型进行评估
evaluation(server, X_test, Y_test)
del X_test,Y_test

INFO:root:Evaluation:
Loss: 0.05720565840601921
Accuracy : 0.9851346611976624

INFO:root:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99    454620
           1       0.99      0.39      0.56       393
           2       1.00      0.99      0.99     25606
           3       0.86      0.99      0.92      2058
           4       1.00      1.00      1.00     46215
           5       0.90      0.97      0.93      1100
           6       0.96      0.99      0.97      1159
           7       1.00      1.00      1.00      1588
           8       1.00      1.00      1.00         2
           9       0.00      0.00      0.00         7
          10       0.83      1.00      0.91     31786
          11       0.90      0.99      0.94      1179
          12       0.42      0.95      0.58       302
          13       0.00      0.00      0.00         4
          14       0.20      0.01      0.01       130

    accuracy                           0.99 

In [ ]:
import pandas as pd

# 读取CSV文件
data = pd.read_csv(os.path.join(PROCESSED_DIR_PATH, 'train_MachineLearningCVE.csv'), skipinitialspace=True)

# 按照gender属性分组
groups = data.groupby('Label')

# 遍历每个分组，将数据写入到不同的CSV文件中
for name, group in groups:
    filename = f'{name}.csv'  # 文件名为gender属性值.csv
    file_name = os.path.join(PROCESSED_DIR_PATH, filename)
    group.to_csv(file_name, index=False)  # 将数据写入到CSV文件中，不包含行索引